In [1]:
#安装第三方包
pip install pandas openpyxl numpy pyperclip

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 导入
import pandas as pd
from datetime import datetime
import re
from openpyxl.utils import get_column_letter
from pandas import ExcelWriter
import numpy as np
import pyperclip
import os

In [3]:
# 输入上次和本次采集数据的日期
datetime_last = datetime(2025,7,14) # 上次采集数据的日期
datetime_this = datetime(2025,7,15) # 本次采集数据的日期

# 构造文件路径
def get_file_paths(date):
    date_str = date.strftime('%Y-%m-%d')
    raw_path = f'raw_data/calculate-{date_str}.xlsx'
    save_path = f'raw_data/proceed/calculate-{date_str}.xlsx'
    return raw_path, save_path

# 创建保存目录（如果不存在）
os.makedirs('raw_data/proceed', exist_ok=True)

# 处理单个文件
def process_file(input_path, output_path):
    # 读取文件
    df = pd.read_excel(input_path)

    # 2.1 去除前两列
    df = df.iloc[:, 2:]

    # 2.2 去除第三列中的所有“#”，并转换为数值型以便正确排序
    if df.shape[1] >= 1:
        df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.replace('#', '', regex=False)
        df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN

    # 2.3 根据第三列排序（现在是第一列）
    df = df.sort_values(by=df.columns[0], ascending=True)

    # 2.4 去除重复行
    df = df.drop_duplicates()
    
    # 2.5 将最后一列中的空值替换为 0
    if df.shape[1] >= 1:
        df.iloc[:, -1] = df.iloc[:, -1].fillna(0)
    
    # 保存结果
    df.to_excel(output_path, index=False)

# 处理两个文件
for dt in [datetime_last, datetime_this]:
    input_file, output_file = get_file_paths(dt)
    process_file(input_file, output_file)

print("文件处理完毕。")


C:\Users\19872\AppData\Local\Temp\ipykernel_6048\1595213853.py:30: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN


文件处理完毕。


C:\Users\19872\AppData\Local\Temp\ipykernel_6048\1595213853.py:30: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # 转成数字，非法项为NaN


In [4]:
import pandas as pd
from datetime import datetime

# 设置用户名（可自定义）
username = '斯格里芬'

# 定义日期
datetime_last = datetime(2025, 7, 14)
datetime_this = datetime(2025, 7, 15)

# 构造文件路径
def get_file_path(date):
    date_str = date.strftime('%Y-%m-%d')
    return f'raw_data/proceed/tongji-{date_str}.xlsx'

# 读取两个文件
df_last = pd.read_excel(get_file_path(datetime_last))
df_this = pd.read_excel(get_file_path(datetime_this))

# 按用户名是否相同进行分组
def split_by_username(df, username_column='author'):
    if username_column not in df.columns:
        raise ValueError(f"列 '{username_column}' 不存在，请检查 Excel 文件。")
    return df[df[username_column] == username], df[df[username_column] != username]

df_last_same, df_last_diff = split_by_username(df_last)
df_this_same, df_this_diff = split_by_username(df_this)

# 统一打印分析报告
print("\n===== 分析报告 =====")
print(f"从{datetime_last.date()} 到{datetime_this.date()}")

# 分析楼主（用户名相同）
rows_same_last = len(df_last_same)
rows_same_this = len(df_this_same)
recommend_same_last = df_last_same.iloc[:, -1].sum()
recommend_same_this = df_this_same.iloc[:, -1].sum()

print(f"[更新] 楼主更新了 {rows_same_this - rows_same_last} 次。")
print(f"[推荐] 增加了 {recommend_same_this - recommend_same_last} 推荐。")

# 查找楼主推荐增加的相同行
def get_changed_recommend(df1, df2):
    if df1.empty or df2.empty:
        return []
    index_cols = df1.columns[:-1]
    df1_indexed = df1.set_index(list(index_cols))
    df2_indexed = df2.set_index(list(index_cols))
    common_index = df1_indexed.index.intersection(df2_indexed.index)
    if common_index.empty:
        return []
    last1 = df1_indexed.loc[common_index].iloc[:, -1]
    last2 = df2_indexed.loc[common_index].iloc[:, -1]
    changed = last1 != last2
    changed_rows = common_index[changed]
    return [(list(idx), last1.loc[idx], last2.loc[idx]) for idx in changed_rows]

changed_main = get_changed_recommend(df_last_same, df_this_same)
if changed_main:
    print("[推荐增加] 发生推荐变化的楼主楼层：")
    for idx, old, new in changed_main:
        print(f"  第 {idx} 楼增加了{new-old}推荐")
else:
    print("[推荐增加] 所有楼层都没有推荐增加。")

# 分析回复（用户名不同）
rows_diff_last = len(df_last_diff)
rows_diff_this = len(df_this_diff)
recommend_diff_last = df_last_diff.iloc[:, -1].sum()
recommend_diff_this = df_this_diff.iloc[:, -1].sum()

print(f"[回复] 增加了{rows_diff_this - rows_diff_last}回复。")
print(f"[回复推荐] 回复增加了 {recommend_diff_this - recommend_diff_last} 推荐。")

changed_reply = get_changed_recommend(df_last_diff, df_this_diff)
if changed_reply:
    print("[推荐增加] 发生推荐变化的回复楼层：")
    for idx, old, new in changed_reply:
        print(f"  第 {idx} 楼增加了{new-old}推荐")
else:
    print("[推荐增加] 所有回复都没有推荐增加")



===== 分析报告 =====
从2025-07-14 到2025-07-15
[更新] 楼主更新了 0 次。
[推荐] 增加了 0 推荐。
[推荐增加] 所有楼层都没有推荐增加。
[回复] 增加了0回复。
[回复推荐] 回复增加了 0 推荐。
[推荐增加] 所有回复都没有推荐增加
